# Test load asin2categoryfiltered

In [ ]:
import json 
asin2cat_filter = dict()
with open("../asin2categoryfiltered.json") as f:
   raw_data = json.load(f)
   for key, value in raw_data.items():
     asin2cat_filter[key] = value

print(f"Loaded {len(asin2cat_filter)} ASIN to Category mappings from filtered data.")